In [ ]:
! pip install vaderSentiment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 1.9 MB/s eta 0:00:00


In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
vs = SentimentIntensityAnalyzer()

In [ ]:
text = 'The'
vs.polarity_scores(text)

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

In [ ]:
# web scraping

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os

In [ ]:
url='https://inshorts.com/en/read/technology'
news_data = []
news_category=url.split('/')
news_category
data = requests.get(url)
soup=BeautifulSoup(data.content)
print(soup)

<!DOCTYPE html>
<html lang="en"><head><meta charset="utf-8"/><title>Short English &amp; Hindi News|Latest News, India News, Current Affairs|Inshorts</title><link href="https://ajax.googleapis.com" rel="dns-prefetch"/><link href="https://assets.inshorts.com" rel="dns-prefetch"/><link href="https://static.inshorts.com" rel="dns-prefetch"/><link href="https://cdn.ampproject.org" rel="dns-prefetch"/><meta content="#ffffff" name="theme-color"/><link href="/assets/images/favicon.png" rel="shortcut icon" type="image/x-icon"/><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="width=device-width,initial-scale=1" name="viewport"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><link href="/assets/images/logo_192.png" rel="icon" sizes="192x192"/><link href="https://inshorts.com" rel="canonical"/>
<link href="https://inshorts.com" rel="alternate"/>
<meta content="read latest news with inshorts in less than 60 words related to business sports bollywood and tech

In [ ]:
urls=['https://inshorts.com/en/read/politics',
      'https://inshorts.com/en/read/world',
      'https://inshorts.com/en/read/sports']
def build_dataset(urls):
  news_data=[]
  for url in urls:
    news_category=url.split('/')[-1]
    news_category
    data = requests.get(url)
    soup=BeautifulSoup(data.content)
    news_articles=[{'news_headline':headline.find('span',attrs={"itemprop":"headline"}).string,
                    'news_article':article.find('div',attrs={"itemprop":"articleBody"}).string,
                    'news_category':news_category}
                    for headline,article in zip(soup.find_all('div',class_=["news-card-title news-right-box"]),
                                                soup.find_all('div',class_=["news-card-content news-right-box"]))]
    news_articles = news_articles[0:20]
    news_data.extend(news_articles)
  df = pd.DataFrame(news_data)
  df=df[['news_headline','news_article','news_category']]
  return df

In [ ]:
df=build_dataset(urls)
df.head()

KeyError: ignored

In [ ]:
df.to_csv('news.csv',index=False)

In [ ]:
#preprocessing for textual data
import nltk
nltk.download('stopwords')
stopword_list=nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')
print (len(stopword_list))

In [ ]:
#remove html tags

def html_tag(text):
  soup=BeautifulSoup(text,"html parser")
  new_text=soup.get_text()
  return new_text


In [ ]:
# expand contractions
! pip install contractions

In [ ]:
# expand contractions
import contractions
def con(text) :
  expand=contractions.fix(text)
  return expand
con("Y'all can't fhyds th're, don't")

In [ ]:
#remove special words
import re
def remove_sp(text) :
  pattern = r'[^A-Za-z0-9\s]'
  text=re.sub(pattern,'',text)
  return text
  remove_sp("+++---asdftyubt gghgbjnk@ 5566@")

In [ ]:
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer = ToktokTokenizer()

In [ ]:
def remove_stopwords(text) :
  tokens = tokenizer.tokenize(text)
  tokens=[token.strip() for token in tokens]
  filtered_tokens=[token for token in tokens if token not in stopword_list ]
  filtered_text= ' '.join(filtered_tokens)

In [ ]:
remove_stopwords("the and if, all there be")

In [ ]:
#1. Lower case
#2. HTML tags
#3. Contractions
#4. Special characters
#5. Stopwords

df.news_headline = df.news_headline.apply(lambda x:x.lower())
df.news_article = df.news_article.apply(lambda x:x.lower())

df.news_headline = df.news_headline.apply(html_tag)
df.news_article = df.news_article.apply(html_tag)

df.news_headline = df.news_headline.apply(con)
df.news_article = df.news_article.apply(con)

df.news_headline = df.news_headline.apply(remove_sp)
df.news_article = df.news_article.apply(remove_sp)

df.news_headline = df.news_headline.apply(remove_stopwords)
df.news_article = df.news_article.apply(remove_stopwords)

df.head()


In [ ]:
df['compound'] = df['news_article'].apply(lambda x: vs.polarity_scores(x)['compound'])

In [ ]:
df.head()